In [19]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [20]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4-turbo'
os.environ['SERPER_API_KEY'] = os.getenv("SERPER_API_KEY")

In [21]:
from crewai import Agent, Crew, Task

CrewAI Tools

In [22]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

Creating Agents

In [23]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [24]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [25]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [26]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [27]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

In [28]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [29]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

### Creating the Crew


**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [30]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

2025-01-21 14:54:08,800 - 8355712000 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

- Set the inputs for the execution of the crew.

In [31]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**Note 2**: 
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [32]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
Thought: To start finding a suitable venue in San Francisco for the Tech Innovation Conference, I need to identify venues that can host such events, especially focusing on those which have facilities and an environment conducive to technology and innovation. I should start by searching for venues that are frequently used for tech conferences in San Francisco.

Action: Search the internet
Action Input: {"search_query": "venues suitable for tech conferences in San Francisco"} 


Search results: Title: The 16 Best Conference Venues for Rent in San Francisco, CA
Link: https://www.tagvenue.com/us/hire/conference-venues/san-francisco
Snippet: Tagvenue offers an extensive range of conference venues in San Francisco, making it easy for you to find the perfect one. Select and book your venue today!




> Entering new CrewAgentExecutor chain...

> Entering new CrewAgentExecutor chain...

Thought: To promote the Tech Innovation Conference effectively, I need to gather information about the target audience and potential marketing channels. I should also check the current trends in promoting tech events to ensure our marketing strategy is up-to-date and appealing.

Action: Search the internet
Action Input: {"search_query": "effective marketing strategies for tech conferences 2023"} 


Search results: Title: How to market an event: An actionable guide for 2023 and beyond
Link: https://socio.events/blog/how-to-market-an-event
Snippet: Learn how to successfully market an event with proven tactics, strategic planning, and leveraging event technology for maximum impact.
---
Title: 15 conference marketing ideas for 2024 that really work | Kaltura
Link: https://corp.kaltura.com/blog/conference-marketing/
Snippet: Explore conference marketing benefits and effective strategies, including virtua

In [33]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)


{'address': '747 Howard Street, San Francisco, CA 94103',
 'booking_status': 'Available',
 'capacity': 0,
 'name': 'The Moscone Center'}


In [35]:
from IPython.display import Markdown
Markdown("marketing_report.md")

## Marketing Activities and Attendee Engagement for the Tech Innovation Conference

### Strategic Marketing Plan
- **Objective**: Engage at least 500 potential attendees at the Tech Innovation Conference.
- **Venue**: The Moscone Center, 747 Howard Street, San Francisco, CA 94103.

### Target Audience
- **Business Professionals** in technology and innovation sectors.
- **Students** and **academics** interested in tech advancements.
- **Industry Leaders** and decision-makers looking for networking and collaborations.

### Promotional Tactics
#### Social Media Marketing
- **Platforms**: Focus on LinkedIn for professionals and Twitter for broader tech audience engagement.
- **Content**: Share speaker profiles, session teasers, and testimonials from past events.
- **Hashtag**: Use #TechInnovate2023 for tracking and promotion.

#### Content Marketing
- **Blog Posts**: Write articles on the importance of tech innovation and previews of conference content.
- **Videos**: Share interviews with keynote speakers and teasers of what attendees can expect.

#### Email Marketing
- **Announcements**: Send out initial save-the-dates with teaser content.
- **Follow-ups**: Detailed emails about speakers, sessions, and exclusive content for early registrants.

#### Influencer Collaboration
- **Keynote Speakers**: Leverage the speakers' networks by providing them with promotional materials to share within their circles.
- **Tech Bloggers**: Engage with tech bloggers to share information about the conference.

#### Paid Advertising
- **LinkedIn Ads**: Target ads to professionals in relevant industries.
- **Google Ads**: Use keywords related to technology conferences and innovation to capture relevant traffic.

#### Registration Strategy
- **Early Bird Tickets**: Offer discounted rates for early registrations to generate early interest and cash flow.
- **Regular Updates**: Keep potential attendees engaged with regular updates about the conference additions and highlights.

### Technology Utilization
- **Event App**: Implement an event app for easy access to schedules, live updates, and networking opportunities.
- **Analytics**: Use data from registration and website traffic to tailor marketing strategies and understand attendee preferences.

### Engagement Initiatives
- **Interactive Sessions**: Design workshops and panels that require active participation, encouraging deeper engagement.
- **Networking Opportunities**: Set up networking zones and use the event app for people to connect and schedule meetings.

### Outcome Measurement
- **Registration Tracking**: Monitor sign-ups and attendance rates.
- **Social Media Engagement**: Analyze hashtag usage and engagement rates on promotional posts.
- **Post-Event Surveys**: Gather feedback to measure attendee satisfaction and areas for improvement.

By implementing these strategies, the Tech Innovation Conference is positioned to not only meet but exceed the engagement goal of 500 attendees, ensuring a successful and impactful event.